In [4]:
import numpy as np
import utils  
from node2vec import Node2Vec  # NODE2VEC EMBEDDINGS 
import classifier as clf  # VERTEX CLASSIFIER



ModuleNotFoundError: No module named 'utils'

DOMAIN OF THE DATA

PPI dataset can be model as a graph of 44906 vertexes belong to the training set (split in 20 sub-graphs) and 12038 belong to the test set (split in 4 sub-graphs) each one charecterized by a binary features vectort (belong to (0,1)^50) very sparse and a no fixed number of undirected edges that connect each vertex to the others. We know the labels (belong to (0,1)^122) of the training set and we want to predict them in the test set with the best accuracy (compute by F1 score). 
These are the distributions of the data: 

<img src="FeaturesTES.png" style="height:300px;width400px">
<img src="FeaturesTEScardinality.png" style="height:300px;width400px">
<img src="FeaturesTRS.png" style="height:300px;width400px">
<img src="FeaturesTEScardinality.png" style="height:300px;width400px">
<img src="FeaturesTES.png" style="height:300px;width400px">
<img src="LabelsTRScardinality.png" style="height:300px;width400px">

(sono le 6 immagini delle ditribuzioni dei dati, da visualizzare a 2 per riga)

'Feature10' is costant to zero in each vertex of the dataset, so we delate it because it can’t explain any causal correlation with the labels. More in general the sparse nature of the features show us that to be able to extract attributes from the graph's structural information is essential in this dataset. Per quanto riguarda le labels invece, we can assume that, for the (naive) indipendence between the Nodes and for Central Limit Theorem, the same means of elements belong to each classes si ripresenterà nel test set (questo ci sarà utile più avanti).

In [2]:
# LOADING THE DATA
vertices_train, vertices_test = utils.build_vertices()

X_train_df = utils.build_dataframe(vertices_train, "feature")
X_train_df = X_train_df.drop(['feature_10'], axis=1)  # DROP USELESS FEATURE

X_test_df = utils.build_dataframe(vertices_test, "feature")
X_test_df = X_test_df.drop(['feature_10'], axis=1)  # DROP USELESS FEATURE

labels_df = utils.build_dataframe(vertices_train, "label", preserve_int_col_name=True)

LOADING

Loaded the data, we build the graph 'G' to model the protein-protein interactions and we extract the matrixes of features ‘X’  (belong to (0,1)^n*50) and the matrix of labels ‘labels’ (belong to (0,1)^n*50).

In [3]:
# BUILDING NUMPY MATRICES 
X_train = X_train_df.values
X_test = X_test_df.values
X = np.concatenate((X_train, X_test), axis=0)
labels = labels_df.values


# MASK FOR TRAIN/TEST
train_idx = range(X_train.shape[0])
test_idx = range(X_train.shape[0],X.shape[0])

# BUILDING THE GRAPH
G = utils.build_graph()
G = G.to_directed()  # CONSIDERING THE GRAPH AS UNDIRECTED


EMBEDDING

Any supervised machine learning algorithm requires a set of informative, discriminating, and independent features. In multi-label vertex classification on graphs this means that one has to construct a feature vector representation for each node, called embedding, but there is no straightforward way to encode this high-dimensional, non-Euclidean information into a low-dimensional vector.
A typical solution involves hand-engineering domain-specific features based on expert knowledge but it is onerous and not generalizable. 
An alternative approach is to learn feature representations by solving an optimization problem. The challenge in feature learning is defining an objective function balancing computational efficiency and predictive accuracy. The goal is to optimize this objective function so that geometric relationships in the embedding space reflect the structure of the original graph. 
In particular we assume that two nodes are neighbour in the embedding space if their features are similar, they belong to the same communities (homophily assumpion) and share similar roles (stuctural equivalence assumption). 
For instance, in Figure 1, we observe nodes u and s1 belonging to the same tightly knit community of nodes, while the nodes u and s6 in the two distinct communities share the same structural role of a hub node.

(foto del paper node2vec)

NODE2VEC

In our first approach to the problem we don't consider the matrixes of features and we focus to embed the vertexes through node2vec algorithm with a definition of neighborhood as flexible as possible. Node2vec indeed, is a semi-supervised algorithm designed to natural lenguage processing for scalable feature learning that instead of trying to decode a
deterministic node similarity measure, it optimizes embeddings encoding the statistics of random walks.
For every source node u ∈ V , we define NS(u) ⊂ V as a network neighborhood of node u generated through a neighborhood sampling strategy S.
Given a source node u, we simulate a random walk of fixed length l. Let ci denote the ith node in the walk, starting with c0 = u. Nodes ci are generated by the following distribution: (formula 3.2.1 paper)
where πvx is the unnormalized transition probability between nodes v and x, and Z is the normalizing constant. We set then the unnormalized transition probability to πvx = αpq(t, x), where (formula). The use of two parameters 'p' and 'q' allowed to combine in a single path breadth-first sampling and depth-first sampling. In particular the return parameter 'p' controls the likelihood of immediately reviseting a node in the walk, and the in-out parameters 'q' allowes the search to differentiate between "inward" and "outward" nodes. Here we set p=1 and q=5 to bias the walk close to node t (BFS) to emphasize structural equivalence.

In [7]:
# GENERATING WALKS OVER THE GRAPH
node2vec = Node2Vec(G, dimensions=128, walk_length=12, num_walks=10, p=1, q=5)


Generating walks (CPU: 1): 100%|██████████| 10/10 [04:35<00:00, 29.62s/it]


SKIP-GRAM (da rivedere) (mappi in maniera diversa contest e oggetto?)

We proceed by extending the Skip-gram architecture for natural lenguage processing to the graph. We seek to optimize the following objective function, which maximizes the log-probability of observing a network neighborhood NS(u) for a node u conditioned on its feature representation, given by f:
(formula)

In order to make the optimization problem tractable, we make two standard assumptions:
• Conditional independence. We factorize the likelihood by assuming that the likelihood of observing a neighborhood node is independent of observing any other neighborhood node given the embedding representation of the source:
(formula)


(verifica)
• Symmetry in feature space. A source node and neighborhood node have a symmetric effect over each other in feature space. Accordingly, we model the conditional likelihood of every source-neighborhood node pair as a softmax unit parametrized by a dot product of their embeddings:
(formula)

With the above assumptions, the objective in Eq. 1 simplifies to:
(formula)

We optimize it using stochastic gradient ascent over the model parameters defining the features f.

In [12]:
# FITTING THE SKIP-GRAM MODEL
skipgram = node2vec.fit(window=8)


#  SAVING OUR EMBEDDINGS
embedding_n2v = skipgram[skipgram.wv.vocab]   
np.save('./embedding/embedding_n2v.npy', embedding_n2v)  # SAVING AS  A .NPY FILE

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


CLASSIFICATION 

Once we have embeded each vertex of the graph the problem becames a standard classification. Because besides we don't use yet the matrixes of features we concatenate them with the embeddings just obtained. In general we should combine only the elements of the orginal features and embeddings with high variance and slow correlation with the others elements. However we assume indipendence between these elements of differens nature and because the feature+embedding space isn't too large also a reduction as PCA is not necessary.

To improve the classification instead, through cross validation, we train the model many times subdividing the training set and making the average of the parameters obtained. So we can also compute an accuracy from the training set but it isn't necessary ugual to the test set one because the model can always overfit. 
(classificazione pesata come?)
The last improvement to the classifier is to condider some inner weights to emphasize the wrong classifications.

In [15]:
# X MATRIX FOR CLASSIFICATION
train_embedding_n2v = embedding_n2v[train_idx]  # EXTRACTING TRAINING SET EMBEDDINGS
test_embedding_n2v = embedding_n2v[test_idx]  #  EXTRACTING TEST SET EMBEDDING
X_train_n2v = np.concatenate((X_train, train_embedding_n2v), axis=1)  #  FEATURES + EMBEDDING
X_test_n2v = np.concatenate((X_test, test_embedding_n2v), axis=1)  # FEATURES + EMBEDDINGS


#  TESTING VALIDATION ACCURACY
#clf.validation_accuracy(X_train_n2v, labels)

#  TESTING TRAIN ACCURACY
node2vec_model = clf.fit_model(X_train_n2v, labels)  # RETURNS TRAINED MODEL AND TRAIN ACCURACY


#  MAKING PREDICTIONS
node2vec_pred = node2vec_model.predict_proba(X_test_n2v) > 0.4  # EVALUATING PREDICTIONS
utils.get_results('./results/node2vec_pred.csv',node2vec_pred, X_test_df)  # SAVING RESULTS IN A .CSV FOR KAGGLE



	Hamming accuracy: 0.438
	Accuracy, exact matches: 0.000
	Macro F1 Score: 0.439
	Micro F1 Score: 0.470


NODE2VEC CONCLUSION

The algorithm shows competitive performance for the dataset even if 0.438 of accuracy is not sufficent to be called good classification. In add to the flexible definition of neighborhood other strenghts are that for the stochastic nature it is robust to perturbations in the form of noisy or missing edges. Furthermore computationally, the major phases
of node2vec are trivially parallelizable, and it can scale to large graph.

GRAPH CONVOLUTIONAL NETWORK

Because above we haven't considered the matrixes of features to build the embeddings, now we try to encode both graph structure and features of nodes through a Graph Convolutional Network, an other semi-supervised algorithm more general. 
Following the idea of T.N. Kipf and M.Wallington we encode the graph structure directly using a neural network model f(X, A) and train on a supervised target for all nodes with labels, thereby avoiding explicit graph-based regularization in the loss function that need the assumpion that "connected nodes share similar labels".

In [6]:
import torch
from gcn.model import GCN 
from train  import train_model 


Primarily we extract the binary adjacency matrix 'A' from the graph, that is symmetrycal because the edges are undirected. Then, without delate the autolaps, we add to 'A' lambda-times the identity matrix 'I' to highlight the importance of own original features in the embeddigs (sum rule), although the original features are very sparse. We also define matrix of degrees Dtilde as (formula) for the Mean Rule. What we want to do is a spectral convolution on the graph and we use Dtilde to normalize the feature rappresentation. Indeed, a propagation rule without a normalization implies that nodes with large degrees will have large values in their features, while nodes with small degrees will have small values.

In [11]:
# BUILDING ADJ MATRIX FOR GCN
A = utils.adjacency_matrix_GCN(G, 1)  


# DEFINING OUR PARAMETERS
n_features = X.shape[1]
n_classes = labels.shape[1]
n_hidden = 32  #  NUMBER OF HIDDEN PARAMETERS IN OUR NET


PROPAGATION RULE

We can so define the propagation rule:
(formula se possibile con piccola dimostrazione dello sviluppo di chebishev)
H(o)=X
H(l+1)= ...
Fixed the number of layers and their dimensions, through a neaural network we build the embeddings of the nodes (that directly rapresent the labels). With this aim we reasearch the best values of the matrixes of weights 'W' that minimize the cross entropy error, training the model with the backpropagation rule. On the other hand, to optimize the number of layers is an optimization problem too complex, so we set 2 hidden layers of 16 neurons each one in a heurstic way. This means that each the embedding of each node combine the feature of his neighbor untill the second order. Finally, to reduce the large numbers of parameters of the matrixes of weghts and prevent neural network from overfitting we apply Dropout: the key idea is to randomly drop units, along with their connections, from the neural network during training. 
(immagine dropout)

In [8]:
# CREATING AND TRAINING OUR MODEL

gcn_model = GCN(n_features, n_hidden, n_classes, dropout=0.5)
embedding_gcn = train_model(gcn_model, X, A, labels, train_idx, epochs=50, lr=0.005, wd=5e-3)


100%|██████████| 50/50 [01:20<00:00,  2.13s/it]


In [11]:
#  EXTRACTING EMBEDDINGS 
train_embedding_gcn = embedding_gcn[train_idx]
test_embedding_gcn = embedding_gcn[test_idx]


#  TESTING TRAIN ACCURACY (SIGMOID TRESHOLD SET AT 0.4 )
gcn_train_pred = torch.sigmoid(train_embedding_gcn).detach().numpy() > 0.4 
utils.get_score(gcn_train_pred, labels)  # HAMMING ACCURACY, F1-MICRO, F1-MACRO

#  SAVING THE EMBEDDING
np.save('./embedding/embedding_gcn.npy', embedding_gcn.detach().numpy())  # .NPY FILE



	Hamming accuracy: 0.710
	Accuracy, exact matches: 0.000


/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


	Macro F1 Score: 0.195
	Micro F1 Score: 0.506


In [12]:
#  MAKING OUR PREDICTION
gcn_test_pred = torch.sigmoid(test_embedding_gcn).detach().numpy() > 0.5  # PREDICTIONS ON TEST SET
gcn_test_pred = utils.a_third_law(labels,gcn_test_pred) # 0.475 accuracy
utils.get_results('./results/gcn_pred.csv',gcn_test_pred, X_test_df)  # SAVING RESULTS IN A .CSV 


Given each embedding belong to R^122 we map it in (0,1)^122 through the sigmoid function and activate it to obtain label prediction. How we could expected, for the generality and structural complexity of the model the accuracy increase. However this increase is minimal and the score is not sufficient yet to be called good classification.
At this point our suspect increase: Are our sparse features belong to (0,1)^50 adequately rappresentative to this task of multilabel vertex classification?
Before to give an answer we try to implement also GraphSAGE and GAT, others two state-of-the-art algorithms to this task.

GraphSAGE

Aiming to make our model richer and richer, from the idea of GCN, we develop now GraphSAGE, a general inductive framework that leverages node feature information to efficiently generate node embeddings for previously unseen data. 
Although inductive capability isn't necessary for our problem we highlight that it is essential for high-throughput and production machine learning systems, which operate on evolving graphs and constantly encounter unseen nodes. In particular could became really usefull if our dataset of PPI were continuosly modified. So, instead of training individual embeddings for each node, it learns a function that generates embeddings by sampling and aggregating features from a node’s local neighborhood. Furthermore, as GCN, by incorporating node features in the learning algorithm, it simultaneously learns the topological structure of each node’s neighborhood as well as the distribution of node features in the neighborhood. 

In [13]:
from sage.model import SAGE


The intuition behind GraphSAGE is that at each iteration, nodes aggregate information from their local neighbors, and as this process iterates, nodes incrementally gain more and more information from further reaches of the graph.
First, each node v ∈ V aggregates the representations of the nodes in its
immediate neighborhood, into a single vector. This aggregation step depends on the representations generated at the previous iteration of the outer loop (i.e., k − 1), and the k = 0 (“base case”) representations are defined as the input node features. After aggregating the neighboring feature vectors, GraphSAGE then concatenates the node’s current representation, with the aggregated neighborhood vector, and this concatenated vector is fed through a fully connected layer with nonlinear activation function σ, which transforms the representations to be used at the next step of the algorithm. We adopt the elementwise mean aggreator as e aggregator function.

This Pseudo-code resume and order all the passes of GraphSAGE
(immagine)

In [14]:
#  CREATING THE MODEL
edge_list = torch.LongTensor(utils.edge_list_SAGE())  # list of all the edges
sage_model = SAGE(n_features, n_hidden, n_classes, 0.5)


#  TRAINING THE MODEL 
embedding_sage = train_model(sage_model, X, edge_list, labels, train_idx, epochs=25, lr=0.1,wd=5e-3)
sage_train_pred = torch.sigmoid(embedding_sage[train_idx]).detach().numpy() > 0.5# TRESHOLD ON SIGMOID PROBABILITIES
sage_train_pred = utils.a_third_law(labels,sage_test_pred)
utils.get_score(sage_train_pred, labels)  # HAMMING ACCURACY, F1-MICRO, F1-MACRO


#  SAVING THE EMBEDDING
np.save('./embedding/embedding_sage.npy', embedding_sage.detach().numpy())  # .NPY FILE


100%|██████████| 25/25 [04:31<00:00, 10.88s/it]


	Hamming accuracy: 0.306
	Accuracy, exact matches: 0.000
	Macro F1 Score: 0.445
	Micro F1 Score: 0.468


In [24]:
#  MAKING OUR PREDICTION
sage_test_pred = torch.sigmoid(embedding_sage[test_idx]).detach().numpy() > 0.5  # PREDICTIONS ON TEST SET
sage_test_pred = utils.a_third_law(labels,sage_test_pred) # BEST SAGE 0.477
utils.get_results('./results/sage_test_pred.csv',sage_test_pred, X_test_df)  # SAVING RESULTS IN A .CSV 


Despite the efforts also GraphSAGE performes an accurancy between 0.44 and 0.46 on the test set, virtually identical to the previous alghorithms. There are already too much labels that we aren't able to predict correctly.
Before to implent GAT we try to combine the three methods seen untill now to combine the respective strenghts. To avoid to connect an eccessive number of features and extend too much the hypotesis space, in these new embeddings we try to consider only the directions of maximum variance (through PCA).

In [ ]:
Y = np.concatenate((embedding_n2v,embedding_sage),axis=1)  # N2V EMBEDDING concatenated SAGE EMBEDDING
clf.validation_accuracy(Y[train_idx],labels)

In order to understand the real efficacy of these concatenations we evaluete these new embeddings through a cross validation on the training set because it is more rappresentative than the test set and don't lead us to make considerations only to force to improve the accuracy of the test set. However either GraphSAGE+Node2Vec (around 0.425 of micro), either GCN+Node2Vec get worse the accuracy obained with the singles algorithm.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 64)
Y_pca = pca.fit_transform(Y)  #  250 --> 64
print('Explained variance: ',np.sum(pca.explained_variance_ratio_))  #  GOOD EXPLAINED VARIANCE
clf.validation_accuracy(Y_pca[train_idx],labels)     

Even the principal compenent analysis, that map these new embeddings of around 250 features in a low-dimensional space that consider only the 64 directions of maximum variance, don't improve our predictions.
This isn't a problem of overfitting.

GAT

A last resort for our multi-label vertex classification problem is to implement GAT. We read that it can performe drastically better than each approaches seen untill now if the features are sparse. In particoular we read in this paper (linkare graph attention network) that GAT can performe an accuracy of more than 0.973 in a dataset of protein-protein interactions with the same dimension of th our. We don't understand yet how could be possible but we continue our research.

In [8]:
from gat.ppi import PPI
from gat.model import GAT
from gat.training import train,test
from torch_geometric.data import DataLoader


GAT is a novel neural network architectures that operates on graph-structured data, leveraging masked self-attentional layers to address the shortcomings of prior methods based on graph convolutions or their approximations.
The idea is to compute the hidden representations of each node in the graph, by attending over its neighbors, following a self-attention strategy. The attention architecturehas several interesting properties: the operation is efficient, since it is parallelizable across nodeneighbor pairs; it can be applied to graph nodes having different degrees by specifying arbitrary
weights to the neighbors; and, as GraphSAGE, the model is directly applicable to inductive learning problems, including tasks where the model has to generalize to completely unseen graphs.

In [9]:
#  LOADING DATASETS 
train_dataset = PPI('./gat',split='train')
test_dataset = PPI('./gat',split='test')
valid_dataset = PPI('./gat',split='valid')  #  LET'S KEEP SOME TRAINING SAMPLES FOR VALIDATION..


#  BATCH TRAINING.. 
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

A Graph Attention Network is composed by many Graph attentional Layers. Each one map a set of node features in a new set of features (potentially of different cardinality). In order to obtain sufficient expressive power to transform the input features into higher-level features, as an initial step, a shared learnable linear transformation, parametrized by a weight matrix, is applied to every node (formula). Then, a shared attentional mechanism 'a' computes attention coefficients that indicate the importance of node j's features to node i. This idea self-attention is a sort genaralization of the GCN algorithm were instead each node has the same influence to its neighbors.  Actually we have already used a sort of attentional coefficent in the GCN lambda when we added lambda-times the identity matrix to the adjacency matrix to emphasize the importance of autolap. 
To make coefficients easily comparable across different nodes, we normalize them across all choices of j using the softmax function. (formula)
Once obtained, the normalized attention coefficients are used to compute a linear combination of the features corresponding to them, to serve as the final output features for every node. (formula)
To stabilize the learning process of self-attention, we also extend the mechanism to employ multi-head attention (formula).

For the inductive learning task, we apply a three-layer GAT model. Both of the first two layers consist of K = 4 attention heads computing F0 = 256 features, followed by an ELU nonlinearity. The final layer is used for (multi-label) classification:
K = 6 attention heads computing 122 features each, that are averaged and followed by a logistic sigmoid activation.
Although the algorithm is parellelizable without a GPU our calculators aren't able to train the model. Till 500 minutes are necessary for a single epoch. So we rent a Tesla K80 and we compute all the optimization in only few minutes.<b>
Trying to improve even more the accurancy, with our simple statistical considerations we impose that the most probable classes of labels are one for all the test set. Even if this kinds of considerations will never lead us to a perfect prediction but at the leatest to a good esteem, however this addition improve here the accuracy of around 0.02.

In [ ]:
#  BUILDING AND TRAINING OUR MODEL
model = GAT(n_features,n_classes)
for epoch in range(1, 60):
    loss = train(model,train_loader) 
    acc = test(model,valid_loader) 
    print('Epoch: {:02d}, Loss: {:.4f}, Acc: {:.4f}'.format(epoch, loss, acc))

In [17]:
#  MAKING PREDICTIONS
model.cpu()
for data in test_loader:    
    out = model(data.x,data.edge_index)
    
pred = out.float().cpu() > 0
pred = utils.a_third_law(labels,pred)

utils.get_results('./results/gat_pred.csv',pred, X_test_df)  # SAVING RESULTS IN A .CSV 

Oooooooo... 0.5207!!! This is the best score obtained until now but it is drammatically low again. How is possible to obtain at least the same F1 score of which the papers talking about? We are wary so we compare our codes and dataset with their. It turns out that our dataset is really similar, but different. Each dimension is ugual, also the labels matrix and adjacency matrix are the same; however the features are different: instead to be binary, their features matrix  $\in [0,1]^{50}$ (it's continuos instead discrete). Continuing with the comparison we infer that our features matrix is just their one activated with a non-liner function (probabily a simple step function). <br><br>
Finally we can set our mind at rest: <b>our data is too less rappresentatitive to obtain a good accurancy</b> as the same algorithms can obtain with similar problems. This doent's mean that each embeddings that we have built until now is useless, conversely each one is a simpler structure where we store better the sparse informations of the dataset. However if the data are insufficient to multi-label vertex classification, also the best embedding can't add new informations.
To show the little worth of our predictions, we show now an our simple statistical algorithm that obtain the same accurancy compute with F1 without consider any features and graph.

STUPID STATISTICAL APPROACH

We want to obtain the best accuracy only through statistical considerations on the labels' distributions. Just with these few informations we can't predict if a new vertex has more or less probability than the others vertexes to have one in a single element of its label. We exclude to build randomly our prediction, so only two strategies remain: put one at the same class of each vertex, or put zero.<br>
To understand the best way to combine this two strategies we build the vector $L_{mean}\in [0,1]^{122}$ as follow:<br><br>
$$ L_{mean}(i) = \frac{\mid\{v\mid L_{v,i}=1\}\mid}{n_{train}}$$ <br>
$L_{mean}$ rapresents the mean of each label class in the training set. As we saw initially, for the Central Limit Theorem, each one of these values should tend to the mean of the same label class in the test set. 

In [ ]:
lmean = utils.get_lmean(labels)    
lmean = utils.sort_lmean(lmean)  #  DESCENDING ORDER 
plt.plot(lmean)

The idea is to force one to a number $s$ of label classes $i$ if $L_{mean}(i)$ is 'big enough'. It remains to define what 'big enough' means. We reorder the element of $L_{mean}$ from the bigger to the smaller and we evaluete $V(s)$, an our approximation of $F1$ for this algorithm.

$$V(s) = 2\cdot \frac {\sum_{i=1}^{s} L_{mean}(i)}{\sum_{i=1}^{122} L_{mean}(i) + n_{train}\cdot s} \simeq F1(s)$$

The variable $s$  goes from $1$ to $122$ and expresses the number of the classes that we activate, starting from that one with bigger mean in the training set. 

In [ ]:
A = 0
k = 0
f1_values = np.zeros(lmean.shape[0])
for i in range(lmean.shape[0]):
    k += lmean[i][0]
for s in range(lmean.shape[0]):
    A += lmean[s][0]
    f1 = 2 * A / (k+ntr*(s+1))
    f1_values[s] = f1

Valuating the function $V$ in all his domain $\{1,2,...,122\}$ and considering the point of its maximum we find the value of $s$ that maximize our strategy and an esteem of its relative F1 score:<br>
$$s_{max}=57$$ <br>
$$V(s_{max})=0.527$$ <br>
The following plot rapresents the trend of $V(s)$.

In [ ]:
%matplotlib inline
plt.plot(f1_values)

In [ ]:
mask = lmean_f[:np.argmax(f1_values),1]
pred = np.zeros((X_test.shape[0],labels.shape[1]))

for col in range(labels.shape[1]):
    for i in range(np.argmax(f1_values)):
        if(mask[i] == col):
            pred[:,col] =1 
            
utils.get_results('./results/stat.csv',pred, X_test_df)  # SAVING RESULTS IN A .CSV  

We evaluate this prediction also on the test set and we obtain an accuracy of $0.485$ in line with our prediction.

This value is bigger than accuracy of GCN, Node2Vec and GraphSAGE and it is obtained without consider any features and graph, and with a costant prediction for each element of the test set. This result allow us to  emphasize a last important conlusion. <br>
$$Machine\space Learning\space is\space not\space magic!$$
If the data aren't enough, we can't expect certain performance also from the best machine learning algorithm for that specific task. Furthermore it is also possible that, in this cases, statistic alone performes better than state-of-the-art algorithm unable to train correctly.